<h2> Информационный поиск. HW2 </h2>
<h2> Валерий Дужик </h2> 

Я сохранил вывод crawler'а в crawled.json, и теперь распаршу. Проиндексирую данные по каждому из урлов, чтобы в граф не попали ребра без вершин + чтобы каждый Node задавался просто урлом, а не всеми данными. 


In [51]:
import json
data = []
with open('crawled.json') as f:
    content = f.read()
    data = json.loads(content)
    
index = dict()
for doc in data:
    index[doc['url']] = {
        'title' : doc['title'],
        'snippet' : doc['snippet']
    }

len(index)

19959

Заиспользую networkx. Построю граф и расчитаю PR для всех страниц.

In [52]:
import networkx as nx

G=nx.Graph()
for doc in data:
    for link in doc['links']:
        if link in index:
            G.add_edge(doc['url'], link)
            
alphas = [0.3, 0.5, 0.85, 0.95]
prs = { a : nx.pagerank(G, alpha=a, max_iter=1000) for a in alphas }

Ну а теперь выведу результаты, и сравню их для разных значений параметра <i>alpha<i>

In [46]:
def get_top(pr, top):
    docs = []
    for url in pr:
        docs.append((url, pr[url]))
    docs.sort(key=lambda x : x[1], reverse=True)
    return [ {
        'PR' : doc[1],
        'title' : index[doc[0]]['title'],
        'url' : doc[0],
        'snippet' : index[doc[0]]['snippet']
    } for doc in docs[:top] ]

def print_top(top):
    for doc in top:
        print(u'{} <{}>\n{}\n{}\n'.format(doc['title'], doc['PR'], doc['url'], doc['snippet']))

In [47]:
print_top(get_top(prs[0.85], 10))

Paul Egede <0.015986652222>
https://fr.wikipedia.org/wiki/Paul_Egede
Paul ou Poul Egede est un missionnaire danois né le 9 septembre 1708 à Kabelvåg, dans les îles Lofoten, et mort le 6 juin 1789 à Copenhague....

Alexandra Grimal <0.0121599927777>
https://fr.wikipedia.org/wiki/Alexandra_Grimal
Alexandra Grimal, née le 28 mai 1980 au Caire, est une autrice-compositrice-interprète française de musique contemporaine. Saxophoniste et chanteuse, elle se produit dans des projets interdisciplinaires comme le théâtre et la danse contemporaine....

Anselm Franz <0.0121599927777>
https://en.wikipedia.org/wiki/Anselm_Franz
Dr. Anselm Franz (January 21, 1900—November 18, 1994) was a pioneering Austrian jet engine engineer known for the development of the Jumo 004, the world's first mass-produced turbojet engine by Nazi Germany during World War II,[1] and his work on turbosha...

Judy Chicago <0.0121599927777>
https://fr.wikipedia.org/wiki/Judy_Chicago
Judy Chicago (née Judy Cohen le 20 juillet 19

In [48]:
print_top(get_top(prs[0.95], 10))

Paul Egede <0.016457771812>
https://fr.wikipedia.org/wiki/Paul_Egede
Paul ou Poul Egede est un missionnaire danois né le 9 septembre 1708 à Kabelvåg, dans les îles Lofoten, et mort le 6 juin 1789 à Copenhague....

Anselm Franz <0.0123955525726>
https://en.wikipedia.org/wiki/Anselm_Franz
Dr. Anselm Franz (January 21, 1900—November 18, 1994) was a pioneering Austrian jet engine engineer known for the development of the Jumo 004, the world's first mass-produced turbojet engine by Nazi Germany during World War II,[1] and his work on turbosha...

Athabaskan languages <0.0123955525726>
https://en.wikipedia.org/wiki/Athapascan_languages
Athabaskan or Athabascan (also Dene, Athapascan, Athapaskan) is a large family of indigenous languages of North America, located in western North America in three groups of contiguous languages: Northern, Pacific Coast and Southern (or Apachean). Kari and...

Agriculture biologique <0.0123955525726>
https://fr.wikipedia.org/wiki/Agriculture_biologique
L’agricu

In [49]:
print_top(get_top(prs[0.3], 10))

Paul Egede <0.0121803283333>
https://fr.wikipedia.org/wiki/Paul_Egede
Paul ou Poul Egede est un missionnaire danois né le 9 septembre 1708 à Kabelvåg, dans les îles Lofoten, et mort le 6 juin 1789 à Copenhague....

Alexandra Grimal <0.0102568308333>
https://fr.wikipedia.org/wiki/Alexandra_Grimal
Alexandra Grimal, née le 28 mai 1980 au Caire, est une autrice-compositrice-interprète française de musique contemporaine. Saxophoniste et chanteuse, elle se produit dans des projets interdisciplinaires comme le théâtre et la danse contemporaine....

Judy Chicago <0.0102568308333>
https://fr.wikipedia.org/wiki/Judy_Chicago
Judy Chicago (née Judy Cohen le 20 juillet 1939 à Chicago dans l'Illinois) est une artiste féministe américaine....

Apennin central <0.0102568308333>
https://fr.wikipedia.org/wiki/Apennin_central
L'Apennin central est la subdivision de la chaîne des Apennins située au cœur de la péninsule en Italie centrale. Il est relié à l'Apennin septentrional dont il est séparé, au nord,

In [50]:
print_top(get_top(prs[0.5], 10))

Paul Egede <0.0138916015625>
https://fr.wikipedia.org/wiki/Paul_Egede
Paul ou Poul Egede est un missionnaire danois né le 9 septembre 1708 à Kabelvåg, dans les îles Lofoten, et mort le 6 juin 1789 à Copenhague....

Alexandra Grimal <0.0111124674479>
https://fr.wikipedia.org/wiki/Alexandra_Grimal
Alexandra Grimal, née le 28 mai 1980 au Caire, est une autrice-compositrice-interprète française de musique contemporaine. Saxophoniste et chanteuse, elle se produit dans des projets interdisciplinaires comme le théâtre et la danse contemporaine....

Judy Chicago <0.0111124674479>
https://fr.wikipedia.org/wiki/Judy_Chicago
Judy Chicago (née Judy Cohen le 20 juillet 1939 à Chicago dans l'Illinois) est une artiste féministe américaine....

Apennin central <0.0111124674479>
https://fr.wikipedia.org/wiki/Apennin_central
L'Apennin central est la subdivision de la chaîne des Apennins située au cœur de la péninsule en Italie centrale. Il est relié à l'Apennin septentrional dont il est séparé, au nord,

Как видно, выдачи получились очень похожими.

Забавно, что начинал я с урлов ['https://en.wikipedia.org/wiki/Minsk', 'https://en.wikipedia.org/wiki/San_Francisco', 'https://en.wikipedia.org/wiki/Hahn'], а закончил, по большей части, во французской вики :)